## **Order of steps taken:**
* **1. Importing the data**
* **2. Removing missing values**
* **3. Making the data containing urls wide**
* **4. Stacking all urls**
* **5. Checking which of the urls need to be extended**
* **6. Getting the domains from urls that don't need to be exetended**
* **7. Saving  long urls with extracted domains**
* **8. Expanding the urls and saving them in folder with long urls**
* **9. Repeating the process until no more urls are being expanded**
* **10. Attaching the expanded urls and extracted domains to the data**

In [1]:
pip install urlexpander

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 2.0 MB/s 
  Created wheel for urlexpander: filename=urlexpander-0.0.37-py3-none-any.whl size=11114 sha256=125a443b9ca91e4f3bdf4c9856c230370b23166ac84d62f1ba5ae10c7f39d6a7
  Stored in directory: /root/.cache/pip/wheels/e0/81/a5/a4dc0253cce05f91641175f336e7664ff5c7d94491fe1cd8bf
  Created wheel for unshortenit: filename=unshortenit-0.4.0-py3-none-any.whl size=12418 sha256=6524209f562042da0fe7aa828a1c99290d86f50fe2d8eb6e6aa1fac1ff4d8818
  Stored in directory: /root/.cache/pip/wheels/3f/5a/2b/6e7e9c8217eba210a0281fd574f25f326941f124a149f09023
Successfully built urlexpander unshortenit


In [72]:
import glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urlexpander
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
from ast import literal_eval
import time

pd.options.mode.chained_assignment = None

# path to folder where the data is located
PATH = "/content/drive/MyDrive/url_expander/"   # IMPORTANT: change the path to the file (I would suggest to create a folder for the process)

# path to file
PATH_FILE = os.path.join(PATH, f"sample_500k.csv")

# path to file for saving results (NOTE: it creates it, so no need to worry)
print("Will create new folder 'expanded_urls' to hold the results")
os.makedirs(f"{PATH}expaded_urls")
PATH_SAVING = os.path.join(PATH, f"expaded_urls")

Will create new folder 'expanded_urls' to hold the results


In [73]:
%%time

# function to deal with literal_eval errors
def eval_test(value):
  '''
  Doctring: the function will be used when literal_eval throws an error in the batch
  '''
  try:
    return literal_eval(value)
  except:
    return list(value.split(" "))



# steps 1-4
def get_urls_ready(df):
    '''
  Docstring: 
    1. removing missing values in column url
    2. cleaning the values (making each value a list with url(s))
    3. expanding values (creating a column for each url based on the number of existing urls in the list made above)
    4. stack everything into one column and create a column with the original index  
    5. return dataframe with columns (url_index = origianl url index; url = url)
    '''
    clean = df[df.outlinks.notna()]   # <-- 1
    try:    # <-- 2
        clean_urls = clean.outlinks.progress_apply(lambda x: literal_eval(x))
    except: 
        print("ERROR: Value Evaluation Using Test Function")
        clean_urls = clean.outlinks.progress_apply(lambda x: eval_test(x))    # <-- function eval_test  
  
    clean["clean_urls"] = clean_urls

    split_urls = pd.DataFrame(clean.clean_urls.to_list(), index = clean.index)    # <-- 3
    split_urls_ready = split_urls.set_index(split_urls.index).agg(list, 1).explode().to_frame("urls").dropna().reset_index(level = 0)  # <-- 4
    return split_urls_ready.rename(columns = {"index": "url_index"})  # <-- 5



# steps 5-6
def get_short(df):
    '''
  Docstring: identifying long and short urls
    1. check if url of short: creating new column "is_short" (True = shortened; False = long)
    2. get domains from long urls
    3. return dataframe with out the column "is_short"
    '''
    clean = get_urls_ready(df)  # <- function get_urls_ready
    clean["is_short"] = clean.urls.progress_apply(lambda x: True if (urlexpander.is_short(x)) else False)  # <-- 1
    clean["domain_extracted"] = clean[clean.is_short == False].urls.progress_apply(lambda x: urlexpander.get_domain(x))   # <-- 2
    return clean.drop("is_short", axis = 1)

# step 7
df = pd.read_csv(PATH_FILE)
result = get_short(df)
result[result.domain_extracted.notna()].to_csv(os.path.join(PATH_SAVING, f"long_urls.csv"), index=False)
df_short_urls = result[result.domain_extracted.isna()].loc[:, result.columns != "domain_extracted"]


# steps 8-9
def get_domain(df_short):
  '''
  Dosctring:
     Function to expand urls and get domains 
     Returning a dataframe with columns (url_index, original_url, expanded_url, domain_extracted)
  '''
  resolved_links = urlexpander.expand(df_short.urls, chunksize=1280, n_workers=64, cache_file='temp.json', verbose=0)  
  df_short["resolved_links"] = resolved_links
  df_short["domain_extracted"] = df_short.resolved_links.progress_apply(urlexpander.get_domain)
  return df_short


def check_domain(df):
  '''
  Docstring: 
    Function to check if url are expanded or not
    Return a list of indexes where the url wasn't expanded
  '''

  idxs_lst = []

  for idx in range(len(df)):
    if df.domain_extracted.iloc[idx] in df.urls.iloc[idx]:
      idxs_lst.append(df.url_index.iloc[idx])
  return idxs_lst


def expand_extract(df_short):
  '''
  Docstring: 
    1. run get_domain() on short urls
    2. check urls that didn't expand with check_domain()
    3. repeat process untill no urls are expanded
    4. save not expanded urls as "unexpandable_urls"
  '''

  df_short.reset_index(drop=True, inplace=True)
  file_number = len(os.listdir(PATH_SAVING))

  resolved_urls = 0
  # 1
  df_run = get_domain(df_short)
  # 2
  idxs_lst = check_domain(df_run)

  # separating expanded urls from those that are not
  df_to_save = df_run[~df_run.url_index.isin(idxs_lst)]
  resolved_urls = len(df_to_save)
  df_rerun = df_run[df_run.url_index.isin(idxs_lst)][["url_index", "urls"]]

  # 3 (threshold for when to stop is 0, this can be changed...)
  if resolved_urls != 0:
    df_to_save.to_csv(os.path.join(PATH_SAVING, f"short_file_{file_number}.csv"), index=False)
    expand_extract(df_rerun)
  # 4
  else:
    df_rerun.to_csv(os.path.join(PATH, f"unexpandable_urls.csv"), index=False)  # not sure if you need this...


expand_extract(df_short_urls)

  0%|          | 0/329998 [00:00<?, ?it/s]

ERROR: Value Evaluation Using Test Function


  0%|          | 0/329998 [00:00<?, ?it/s]

  0%|          | 0/368265 [00:00<?, ?it/s]

  0%|          | 0/212048 [00:00<?, ?it/s]

...

ERROR:urllib3.connection:Certificate did not match expected hostname: beat.10ztalk.com. Certificate: {'subject': ((('commonName', '*.ingress-daribow.ewp.live'),),), 'issuer': ((('countryName', 'GB'),), (('stateOrProvinceName', 'Greater Manchester'),), (('localityName', 'Salford'),), (('organizationName', 'Sectigo Limited'),), (('commonName', 'Sectigo RSA Domain Validation Secure Server CA'),)), 'version': 3, 'serialNumber': '3A05ADB27E58BC06B2A747FC788C8B49', 'notBefore': 'May 24 00:00:00 2022 GMT', 'notAfter': 'May 24 23:59:59 2023 GMT', 'subjectAltName': (('DNS', '*.ingress-daribow.ewp.live'), ('DNS', 'ingress-daribow.ewp.live')), 'OCSP': ('http://ocsp.sectigo.com',), 'caIssuers': ('http://crt.sectigo.com/SectigoRSADomainValidationSecureServerCA.crt',)}
ERROR:urllib3.connection:Certificate did not match expected hostname: beat.10ztalk.com. Certificate: {'subject': ((('commonName', '*.ingress-daribow.ewp.live'),),), 'issuer': ((('countryName', 'GB'),), (('stateOrProvinceName', 'Greate

  0%|          | 0/5176 [00:00<?, ?it/s]

ERROR:urllib3.connection:Certificate did not match expected hostname: beat.10ztalk.com. Certificate: {'subject': ((('commonName', '*.ingress-daribow.ewp.live'),),), 'issuer': ((('countryName', 'GB'),), (('stateOrProvinceName', 'Greater Manchester'),), (('localityName', 'Salford'),), (('organizationName', 'Sectigo Limited'),), (('commonName', 'Sectigo RSA Domain Validation Secure Server CA'),)), 'version': 3, 'serialNumber': '3A05ADB27E58BC06B2A747FC788C8B49', 'notBefore': 'May 24 00:00:00 2022 GMT', 'notAfter': 'May 24 23:59:59 2023 GMT', 'subjectAltName': (('DNS', '*.ingress-daribow.ewp.live'), ('DNS', 'ingress-daribow.ewp.live')), 'OCSP': ('http://ocsp.sectigo.com',), 'caIssuers': ('http://crt.sectigo.com/SectigoRSADomainValidationSecureServerCA.crt',)}
ERROR:urllib3.connection:Certificate did not match expected hostname: beat.10ztalk.com. Certificate: {'subject': ((('commonName', '*.ingress-daribow.ewp.live'),),), 'issuer': ((('countryName', 'GB'),), (('stateOrProvinceName', 'Greate

  0%|          | 0/5171 [00:00<?, ?it/s]

0 urls resolved, stopping!!!
CPU times: user 2h 58min 55s, sys: 15min 15s, total: 3h 14min 10s
Wall time: 2h 9min 39s


In [151]:
%%time
# step 10
dfs_lst = []
for file in tqdm(os.listdir(PATH_SAVING)):
  print(file)
  data = pd.read_csv(os.path.join(PATH_SAVING, file))
  dfs_lst.append(data)

# connecting data
df_domains = pd.concat(dfs_lst)
# aggregating domains
df_domains_agg = df_domains[["url_index", "domain_extracted"]].groupby("url_index")["domain_extracted"].agg(",".join)
# aggregating expanded urls
df_urls_agg_expanded = df_domains[df_domains.resolved_links.notna()][["url_index", "resolved_links"]].groupby("url_index")["resolved_links"].agg(",".join)
# aggregating urls that don't need to be expanded
df_urls_agg = df_domains[df_domains.resolved_links.isna()][["url_index", "urls"]].groupby("url_index")["urls"].agg(",".join)

'''
1. expanding data and setting the original index as the index
2. renaming columns: 
    a. domain_1 to domain_n = extracted domain
    b. expanded_url_1 to expanded_url_n = urls that needed extention
    c. not_expanded_url_1 to not_expanded_url_n = urls that didn't need extantion

NOTE BELOW YOU CAN COMMENT OUT WHAT IS NOT NEEDED...
'''

# domains
# 1
df_domains_wide = df_domains_agg.str.split(",",expand=True)
# removing index name
df_domains_wide.index.name = None
# 2
domain_cols = {i: f"domain_{i+1}" for i in list(df_domains_wide.columns)} 
df_domains_wide.rename(columns=domain_cols, inplace=True)

# expanded urls
# 1
df_urls_exp_wide = df_urls_agg_expanded.str.split(",",expand=True)
# removing index name
df_urls_exp_wide.index.name = None
# 2
urls_exp_cols = {i: f"expanded_url_{i+1}" for i in list(df_urls_exp_wide.columns)} 
df_urls_exp_wide.rename(columns=urls_exp_cols, inplace=True)

# urls
# 1
df_urls_wide = df_urls_agg.str.split(",",expand=True)
# removing index name
df_urls_wide.index.name = None
# 2
url_cols = {i: f"not_expanded_url_{i+1}" for i in list(df_urls_wide.columns)} 
df_urls_wide.rename(columns=url_cols, inplace=True)


# saving info extracted to the data
df_final = pd.concat([df, df_domains_wide, df_urls_exp_wide, df_urls_wide], axis=1)
df_final.to_csv(os.path.join(PATH, f"final_result.csv"), index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

long_urls.csv
short_file_1.csv
short_file_2.csv
short_file_3.csv
short_file_4.csv
CPU times: user 14.2 s, sys: 473 ms, total: 14.7 s
Wall time: 15.8 s


In [154]:
df_final = pd.read_csv(os.path.join(PATH, f"final_result.csv"), low_memory=False)

In [158]:
df_final[df_final.domain_9.notna()].T

,244187,429416
Unnamed: 0,244187,429416
timestamp,"2018/03/21, 14:30:55","2018/03/10, 20:59:57"
text,🏦 Earn #Bitcoins Signup to https://t.co/GOcEsA...,Some Major Companies That Accept #Bitcoin : \n...
outlinks,"['https://coinpot.co/', 'https://goo.gl/kP6XQ4...","['http://cheapair.com', 'http://Overstock.com'..."
domains,coinpot.co,cheapair.com
domain_1,coinpot.co,cheapair.com
domain_2,coinpot.co,overstock.com
domain_3,moonbit.co.in,expedia.com
domain_4,moondoge.co.in,egifter.com
domain_5,moonliteco.in,newegg.com
